###ポイントで理解するランダムフォレスト(Random Forests)
https://qiita.com/g-k/items/e7dcc4d2b057dada405c

- Random forests - classification description(https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm)
- StatQuest: Random Forests Part 1 - Building, Using and Evaluating(https://youtu.be/J4Wdy0Wc_xQ)
- StatQuest: Random Forests Part 2: Missing data and clustering(https://youtu.be/nyxTdL_4Q-Q)

####ランダムフォレストの特徴
#####ランダムフォレストは簡単に言うと沢山の決定木を作成してその多数決をとるアルゴリズムです。
#####下記のような特徴があり、非常に優れています

- 精度が非常に良い
- 過学習を抑える効果がある
- 何千もの入力変数を削除せずにそのまま扱うことができる
- 各変数の事前のスケーリングが不要
- 交差検証や別個のテストデータでの検証をせずとも、アルゴリズム内で未知データに対する精度を推定することができる
- 欠落値を推定する方法を持っているため、欠損値の多いデータでも精度を維持することができる
- データ間の関係について情報を得ることができる

####ポイント① ブートストラッピング

- ランダムフォレストは決定木の集合体です。決定木を沢山作って森にしている訳ですが、その1つ1つの木の作り方にポイントがあります。まずは決定木を作成する際の訓練データの選び方です。
- ランダムフォレストでは1つ1つの決定木を作成する際に、全データの中から重複を許してサンプリングを行い、そのデータを決定木を作成する際の訓練データとします。この手法のことをブートストラッピングと呼びます。
- 重複を許してサンプリングを行なっているので、もちろん訓練データの中には同じデータが含まれることもあります

####ポイント② 使用する変数を絞る
- 各決定木におけるノード(分岐点)作成する際に使用する変数を絞るのもポイントです。
- データ全量の中に*p*個の変数があったとした時その全てを各ノードを作成するのに使うのではなく、*m*個の変数をランダムに選んで使用します。個数としては $m = \sqrt{p}$ 程度の数がよく用いられます。

- この工程を繰り返しながら大量の決定木を作成していきます。
- 通常は100本以上の木を作成します。(sklearnのランダムフォレストのデフォルト値は100)
- この工程を繰り返すことで多様な木が作成されますが、その多様さがランダムフォレストをより効果的にしています

####ポイント③ バギング(Bagging)
- ランダムフォレストでは最終的に1本1本の木による多数決の結果を最終的な出力とします
- 各学習器に使う学習用データのをブートストラッピングによって選び、その学習器を予測に用いて最後にアンサンブルする方法をバギング(Bagging)と呼び、ランダムフォレストはその一種です(Baggingはbootstrap aggregatingの略)
- バギング自体には分散を減らし、過剰適合を避ける効果があるため、ランダムフォレスト以外にも様々なところで用いられています

####ポイント④ OOB(out-of-bag)検証
- ブートストラッピング法でデータをサンプリングすると必ずそこに選ばれなかったデータが発生します
- 大体元データの $\frac{1}{3}$ 程度のデータが残るのですが、そのデータのことをOOB(out-of-bag)データと呼びます

- このOOBデータによって未知データに対する精度を交差検証や別個のテストデータを要さず推定することができるのも大きなポイントです
- 作成したランダムフォレストを用いてOOBデータを分類し、その分類の分類誤差を確認します(回帰の場合は最小二乗誤差等の指標を確認)
- 分類であれば、そのような誤分類されたOOBデータの割合をOOBエラー(out-of-bag-error)と呼びます

- このOOBエラーの率に応じて使用する変数の数を調整することで精度の最大化を目指します

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
data = iris['data']
target = iris['target']

X_train, X_test ,Y_train, Y_test = train_test_split(data, target, test_size = 0.7, shuffle = True, random_state = 42)

rf = RandomForestClassifier(oob_score =True)

rf.fit(X_train, Y_train)

print('test_data_accuracy:' + str(rf.score(X_test, Y_test)))
print('oob_data_accuracy:' + str(rf.score(X_train, rf.oob_decision_function_.argmax(axis = 1))))

test_data_accuracy:0.9428571428571428
oob_data_accuracy:0.9777777777777777


###スタッキングによる分類
https://qiita.com/maskot1977/items/de7383898123fa378d86

- 分類モデルの性能を確認するため、乳がんデータを使ってみます

In [2]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

estimators = [
        ('svc', make_pipeline(StandardScaler(), SVC())),
        ('rf', RandomForestClassifier()),
        ('mlp', MLPClassifier(max_iter=10000))
        ]
clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=10000)
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.958041958041958

- 比較として、単独の分類モデルの正解率を計算してみます

In [4]:
make_pipeline(StandardScaler(), SVC()).fit(X_train, y_train).score(X_test, y_test)

0.958041958041958

In [5]:
RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)

0.9790209790209791

In [6]:
MLPClassifier(max_iter=10000).fit(X_train, y_train).score(X_test, y_test)

0.9440559440559441

In [7]:
LogisticRegression(max_iter=10000).fit(X_train, y_train).score(X_test, y_test)

0.958041958041958

###スタッキングによる回帰
https://qiita.com/maskot1977/items/de7383898123fa378d86

- 回帰モデルの性能を確認するため、糖尿病データを使ってみます

In [ ]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.cross_decomposition import PLSRegression

estimators = [
        ('svr', make_pipeline(StandardScaler(), SVR())),
        ('rf', RandomForestRegressor()),
        ('mlp', MLPRegressor(max_iter=10000))
        ]
clf = StackingRegressor(
    estimators=estimators,
    final_estimator=PLSRegression(),
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

- 比較として、単独の回帰モデルののR2値を計算してみます

In [ ]:
make_pipeline(StandardScaler(), SVR()).fit(X_train, y_train).score(X_test, y_test)

In [ ]:
RandomForestRegressor().fit(X_train, y_train).score(X_test, y_test)

In [ ]:
MLPRegressor(max_iter=10000).fit(X_train, y_train).score(X_test, y_test)

In [ ]:
PLSRegression().fit(X_train, y_train).score(X_test, y_test)

###aaa

###機械学習素人のマーケターがKaggleのHousePricesに挑戦してみた（前編）
https://qiita.com/borukke/items/93fb596fef13a8d43589

In [ ]:
import pandas as pd

pd.set_option("display.max_columns" , 200)
pd.set_option("display.max_rows" , 100000)

train = pd.read_csv("/content/house-prices-train.csv")
test = pd.read_csv("/content/house-prices-test.csv")

In [ ]:
#data_description.txtに変数定義一覧が載っている
print(train.head())

In [ ]:
#どの変数で欠損値が発生しているかを一覧で確認
print(train.isnull().sum())

In [ ]:
#説明変数と目的変数に分割
train_X = train.iloc[:,:-1]
train_y = train["SalePrice"]

#「Id」も加工には不要なので削除
train_X = train_X.drop(["Id"] , axis=1)

In [ ]:
#カテゴリー変数の変換処理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

columns = ["MSZoning","Street","Alley","LotFrontage","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence","MiscFeature","SaleType","SaleCondition"]

for col in columns:
    train_X[col] = le.fit_transform(train_X[col].astype(str))

In [ ]:
#ラベルエンコーダーの適用によってMSZoningの値がRL→３に変換されている
train_X.head()

In [ ]:
#MSZoningのユニークな値も確認
train_X["MSZoning"].unique()

In [ ]:
#欠損値の補完
#ラベルエンコーダーによって、欠損値が特定の数字に置換されている

train.isnull().sum()

In [ ]:
# LotFrontage MasVnrArea GarageYrBlt を補完

import matplotlib.pyplot as plt

# グラフを入れる枠の大きさ
plt.figure(figsize=(10, 4))

# １行２列の枠の左側に配置
plt.subplot(1,2,1)
plt.hist(train["MasVnrArea"] , bins=30 , label="MasVnrArea")
plt.legend(loc="best")
print("【MasVnrArea】平均値:{:.2f} 中央値:{:}".format(train["MasVnrArea"].mean() , train["MasVnrArea"].median()))

# １行２列の枠の右側に配置
plt.subplot(1,2,2)
plt.hist(train["LotFrontage"] , bins=30 , label="LotFrontage")
print("【LotFrontage】平均値:{:.2f} 中央値:{:}".format(train["LotFrontage"].mean() , train["LotFrontage"].median()))
plt.legend(loc="best")

In [ ]:
# 中央値補完
train_X["MasVnrArea"] = train_X["MasVnrArea"].fillna(train_X["MasVnrArea"].median())

# 平均値補完
train_X["LotFrontage"] = train_X["LotFrontage"].fillna(train_X["LotFrontage"].mean())

In [ ]:
#GarageYrBltは欠損値を「0」にしても、他のGarage関連の変数と照らし合わせれば「あ、こいつGarage持ってないやつね」ってAIは認識してくれるのでは？という仮説のもと、０で置き換え

train_X["GarageYrBlt"] = train_X["GarageYrBlt"].fillna(0)

In [ ]:
#変数を絞り込む

import seaborn as sns
plt.figure(figsize=(20,15))
sns.heatmap(train.corr() , cmap="Blues" , annot=True)

In [ ]:
#相関係数の大きい変数を降順に出力し、0.3以上の変数のみピックアップ
train.corr()["SalePrice"].sort_values(ascending=False)

In [ ]:
# データフレームに変換し、カラム名をリスト化

import numpy as np
df = pd.DataFrame(train.corr()["SalePrice"].sort_values(ascending=False))
df = df.query("0.3 <= SalePrice < 1.0")
columns_needed = np.array(df.index)
train_X = train_X[columns_needed]

In [ ]:
#テストデータの加工
#トレーニングデータと欠損値の数や欠損している変数が異なる

# テストデータからIDを削除
test_X = test.drop(["Id"] , axis=1)

# カテゴリー変数にラベルエンコーダーを適用
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

columns = ["MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence","MiscFeature","SaleType","SaleCondition"]

for col in columns:
    test_X[col] = le.fit_transform(test_X[col].astype(str))

In [ ]:
# 平均値補完
test_X["LotFrontage"] = test_X["LotFrontage"].fillna(test_X["LotFrontage"].mean())
test_X["BsmtUnfSF"] = test_X["BsmtUnfSF"].fillna(test_X["BsmtUnfSF"].mean())
test_X["BsmtFullBath"] = test_X["BsmtFullBath"].fillna(test_X["BsmtFullBath"].mean())
test_X["GarageArea"] = test_X["GarageArea"].fillna(test_X["GarageArea"].mean())
test_X["TotalBsmtSF"] = test_X["TotalBsmtSF"].fillna(test_X["TotalBsmtSF"].mean())
test_X["MasVnrArea"] = test_X["MasVnrArea"].fillna(test_X["MasVnrArea"].mean())
test_X["BsmtFinType2"] = test_X["BsmtFinType2"].fillna(test_X["BsmtFinType2"].mean())
test_X["BsmtFinSF1"] = test_X["BsmtFinSF1"].fillna(test_X["BsmtFinSF1"].mean())
test_X["BsmtFinSF2"] = test_X["BsmtFinSF2"].fillna(test_X["BsmtFinSF2"].mean())
test_X["BsmtHalfBath"] = test_X["BsmtHalfBath"].fillna(test_X["BsmtHalfBath"].mean())
test_X["GarageCars"] = test_X["GarageCars"].fillna(test_X["GarageCars"].mean())

# ０補完
test_X["GarageYrBlt"] = test_X["GarageYrBlt"].fillna(0)

In [ ]:
#トレーニングデータと同様、相関係数が0.3以上の変数に絞ります
test_X = test_X[columns_needed]

In [ ]:
# ランダムフォレストでモデル構築
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

# パラメータをグリッドサーチ
from sklearn.model_selection import GridSearchCV
parameters = {"n_estimators":[10,30,50,70,100,130], 
              "criterion":["mae","mse"],
              "max_depth":[3,5,7,10,15], 
              "max_features":["auto"], 
              "random_state":[0], 
              "n_jobs":[-1]}

In [ ]:
# グリッドサーチでモデル作成
clf = GridSearchCV(rf , parameters , scoring="neg_root_mean_squared_error" , cv=5)
clf.fit(train_X , train_y)

# ベストパラメータを取得
print(clf.best_estimator_)

In [ ]:
#テストデータを予測

pred_y = clf.predict(test_X)
pred_y

In [ ]:
test["SalePrice"] = pred_y
test[["Id","SalePrice"]].head()

import csv
test[["Id","SalePrice"]].to_csv("submission.csv" , index=False)

###データ解析を行う際のTips ・注意点
https://qiita.com/Shogo-dayo/items/f0bd44793231f37b73c6

####前処理
まず最初にデータ(DataFrame -> df)を読み込んで EDA (探索的データ解析) を行います.

- .head() : データの先頭の表示 , (5)であれば5行抽出, defalt値は5
- .info() : データの要約 (行数、列数、各列の列名、各列に格納されるデータの型 etc....)
- .describe() : データの基礎統計量 (min, max, 25% etc.....)
- .shape[0], .shape[1] : データの行数および列数の確認
- .columns : データのカラム名の取得
- .isnull().sum() : 各列の欠損地の個数確認
- .dtypes : 各列の型タイプを確認

In [ ]:
# import
import pandas as pd
import matplotlib.pyplot as plt    ## for drawing graph 

## load Data
df = pd.read~~~~(csv , json etc...)   

# データの先頭表示
df.head() 

# データの要約表示
df.info()

# データの次元数（何行，何列） 
print('There are {} rows and {} columns in df'.format(df.shape[0], df.shape[1]))

# データのカラム（列）取得
df.columns

# 各列データの欠損値の個数をカウント
df.isnull().sum()

# 各列データの型タイプ確認
df.dtypes

df.columnsでカラム名を取得することができますが、型タイプに合わせたカラム名のリストを保持しておくことで、後々のことを考えておくと利用できる可能性がある。

下記にそのコードを記載しておく。

includeを型タイプ（float64 etc...）に変更することも可能

In [9]:
obj_columns = df.select_dtypes(include=['object']).columns
numb_columns = df.select_dtypes(include=['number']).columns

NameError: ignored

####plot例
#####各列（特徴量）のヒストグラム
- 各列の特徴量をヒストグラム化する。
- しかしこれは数値データのみ取り扱うことができる。
- 文字列データに対しては適用できないため、それは後々記述する。

##### .hist(bins=*, figsize=(,*))
- binsは頻度を分析する階級値の細かさの設定、figsizeは図の大きさの設定
- 他にも多くの引数をしてすることができるので、下記を参照していただきたい。

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

####相関行列のヒートマップ
- 機械学習をする上で、重要になってくるのが各特徴量の相関行列を求めることです。
- その相関行列の可視化を行うためのヒートマップで可視化する

#### .heatmap(corr, annot=bool(True or False), cmap='***')
- corrは相関行列、annotはセルに値を設定、cmapは図のカラーを指定
- 他にも多くの引数をしてすることができるので、下記を参照していただきたい。

https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
import seaborn as sns  ## for drawing graph 
corr = train_df.corr()
corr[np.abs(corr) < 0.1] = 0  ## corr<0.1  =>  corr=0 
sns.heatmap(corr, annot=True, cmap='YlGnBu')
plt.show()

####特徴量の重要度の推定
- どの特徴量が、targetであるSalePricesにとって重要であるかをランダムフォレスト(RandomForest)を用いて算出する
- RandomForestを行うために、まず説明変数と目的変数に分割する必要がある
- 今回はtarget(SalePsices)以外の変数を全て説明変数とする

#### .drop("**", axis=(0 or 1))
- **には使用しない列を指定、axis=0である場合は行、1である場合は列

#### RandomForestRegressor(n_estimators=**)
- n_estimatorsは学習回数

In [ ]:
from sklearn.ensemble import RandomForestRegressor
X_train = df.drop("SalePrices", axis=1)
y_train = df[["SalePrices"]]
rf = RandomForestRegressor(n_estimators=80, max_features='auto')
rf.fit(X_train, y_train)

ranking = np.argsort(-rf.feature_importances_)  ##重要度が高い順に描画するため
sns.barplot(x=rf.feature_importances_[ranking], y=X_train.columns.values[ranking], orient='h')
plt.show()

####各特徴量の分布（外れ値の確認用など）
- 回帰分析などを行う際に、特徴量に外れ値が含まれている場合、モデルの精度に影響しやすいことが言われている
- そのため、モデルの精度向上には外れ値処理が必須であると考えられている
- そのために各特徴量には、どれくらいの外れ値が含まれているかを可視化して確認することがわかりやすい
- ここでは、特徴量の重要度上位30位までのplotを行う

#### .iloc[:,:]
- 行あるいは列を指定することで値を抽出

#### sns.regplot
- 2次元のデータと線形回帰モデルの結果を重ねてplot

In [ ]:
X_train = X_train.iloc[:,ranking[:30]]

fig = plt.figure(figsize=(12,7))
for i in np.arange(30):
    ax = fig.add_subplot(5,6,i+1)
    sns.regplot(x=X_train.iloc[:,i], y=y_train)

plt.tight_layout()
plt.show()

####target（目的変数）が正規分布にしたがっているかどうかの確認
- 今回のデータではtargetはSalePricesである
- 機械学習を行うにあたって、目的変数が正規分布に従っているかどうかはモデルに影響するため重要とされています
- そこでSalePricesの分布を見ていきます
- この図は、縦軸は割合、横軸はSalePricesを示しています

In [ ]:
sns.distplot(y_train, color="red", label="real_value")
plt.legend()
plt.show()

#####目的変数が正規分布に従うように、よく使われる対数変換および差分変換を下記に示します

In [ ]:
y_train2 = np.log(y_train)
y_train2 = y_train2.replace([np.inf, -np.inf], np.nan)
y_train2 = y_train2.fillna(0)
sns.distplot(y_train2, color="blue", label="log_value")
plt.legend()
plt.show()

#対数変換を行うことで正規分布に近い図になる

In [ ]:
y_train3 = y_train.diff(periods = 1)
y_train3 = y_train3.fillna(0)
sns.distplot(y_train3, color="green", label="diff_value")
plt.legend()
plt.show()

#差分変換を行うことで正規分布と言えるような図になる

####どのデータを取り扱う時でも、値に偏りがないかどうかを確認しておいた方が良い

###aaa